# Evaluación de la segmentación

In [1]:
import cv2 as cv
from matplotlib import pyplot as plt

In [2]:
image_10c = cv.imread('im/10c.jpg', cv.IMREAD_GRAYSCALE)
image_10t = cv.imread('im/10t.jpg')
image_all = cv.imread('im/all.jpg', cv.IMREAD_GRAYSCALE)
image_rgb = cv.imread('im/rgb.png', cv.IMREAD_GRAYSCALE)

In [3]:
im = image_rgb
# Los números son las intensidades min y máx para identificar el borde
canny = cv.Canny(im, 25, 75)
cv.imwrite('out/canny.jpg', canny)

True

In [4]:
contornos, _ = cv.findContours(canny, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
out = cv.drawContours(im, contornos, -1, (210,139,38), 2)
print('Número de contornos encontrados: ', len(contornos))
texto = 'Contornos encontrados: '+ str(len(contornos))
cv.putText(out, texto, (10,20), cv.FONT_HERSHEY_SIMPLEX, 0.7,
  (0, 0, 0), 1)
out = cv.cvtColor(out, cv.COLOR_BGR2RGB)
cv.imwrite('out/out.jpg', out)

Número de contornos encontrados:  3


True